In [1]:
import cv2
import json
import numpy as np
from numpy import NaN, Inf, arange, isscalar, asarray, array

In [2]:
CROP_RATIO = 0.75
MIN_MATCH_COUNT = 7

In [3]:
def peakdet(v, delta, x=None):
    """
    Converted from MATLAB script at http://billauer.co.il/peakdet.html

    Returns two arrays

    function [maxtab, mintab]=peakdet(v, delta, x)
    %PEAKDET Detect peaks in a vector
    %        [MAXTAB, MINTAB] = PEAKDET(V, DELTA) finds the local
    %        maxima and minima ("peaks") in the vector V.
    %        MAXTAB and MINTAB consists of two columns. Column 1
    %        contains indices in V, and column 2 the found values.
    %      
    %        With [MAXTAB, MINTAB] = PEAKDET(V, DELTA, X) the indices
    %        in MAXTAB and MINTAB are replaced with the corresponding
    %        X-values.
    %
    %        A point is considered a maximum peak if it has the maximal
    %        value, and was preceded (to the left) by a value lower by
    %        DELTA.

    % Eli Billauer, 3.4.05 (Explicitly not copyrighted).
    % This function is released to the public domain; Any use is allowed.

    """
    maxtab = []
    mintab = []

    if x is None:
        x = arange(len(v))

    v = asarray(v)

    if len(v) != len(x):
        sys.exit('Input vectors v and x must have same length')

    if len(v) < 1:
        sys.exit('Array must have at least 1 element')

    if not isscalar(delta):
        sys.exit('Input argument delta must be a scalar')

    if delta <= 0:
        sys.exit('Input argument delta must be positive')

    mn, mx = v[0], v[0]
    mnpos, mxpos = None, None

    lookformax = True
    maxWidth = 0
    minWidth = 0

    for i in arange(1, len(v)):
        this = v[i]
        if this > mx:
            mx = this
            mxpos = x[i]
            maxWidth += 1
        if this < mn:
            mn = this
            mnpos = x[i]
            minWidth += 1

        if lookformax:
            if this < mx-delta:
                if mxpos != None:
                    """
                        TODO: width here might not be really accurate if there is a 
                        sharp increase from the left, and then a slower drop to the right,
                        the width will still be really small. We need a way to get the width from both 
                        sides of the peak
                    """
                    maxtab.append(
                        (mxpos, mx, findPeakWidth(mxpos, v, mx, findMax=True)))
                mn = this
                maxWidth = 0
                mnpos = x[i]
                lookformax = False
        else:
            if this > mn+delta:
                if mnpos != None:
                    mintab.append(
                        (mnpos, mn, findPeakWidth(mnpos, v, mn, findMax=False)))
                mx = this
                minWidth = 0
                mxpos = x[i]
                lookformax = True

    return array(maxtab), array(mintab)


def findPeakWidth(idx, arr, val, findMax):
    width = 0
    if (findMax):
        # Find the furthest minimum left
        i = idx - 1
        # print(arr[i - 10: i + 10])
        # print(arr[i])
        while (i > 0 and arr[i] > arr[i - 1]):
            width += 1
            i -= 1
        # print('First', i, width)
        # Find the furthest minimum right
        i = idx
        while (i < len(arr) - 1 and arr[i] > arr[i + 1]):
            width += 1
            i += 1
        # print('Sec', i, width)
    else:
        # Find the furthest maximum left
        i = idx
        while (i > 0 and arr[i] < arr[i - 1]):
            width += 1
            i -= 1
        # Find the furthest maximum right
        i = idx
        while (i < len(arr) - 1 and arr[i] < arr[i + 1]):
            width += 1
            i += 1
    return width

def calculateRedColorPercentage(img):
    print('[INFO] detectRedColor()')
#     hsv = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
    plt.show()
    hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
    # Red color range
    RED_COLOR_LOW_HUE_LOWER = np.array(
        [0, 100, 100]
    )

    RED_COLOR_LOW_HUE_UPPER = np.array([10, 255, 255])
    RED_COLOR_HIGH_HUE_LOWER = np.array([160, 150, 100])
    RED_COLOR_HIGH_HUE_UPPER = np.array([179, 255, 255])
    
    lower_red_threshold = cv2.inRange(hsv, RED_COLOR_LOW_HUE_LOWER, RED_COLOR_LOW_HUE_UPPER)
    upper_red_threshold = cv2.inRange(hsv, RED_COLOR_HIGH_HUE_LOWER, RED_COLOR_HIGH_HUE_UPPER)

    red_threshold = cv2.addWeighted(lower_red_threshold, 1.0, upper_red_threshold, 1.0, gamma=0.0)
    plt.imshow(red_threshold)
    plt.show()
    red_color_percentage = cv2.countNonZero(red_threshold) / (red_threshold.shape[0] * red_threshold.shape[1])

    return red_color_percentage

def detectBlood(img, blood_threshold=0.25):
    print('[INFO] detectBlood()')
    red_color_percentage = self.calculateRedColorPercentage(img)
    return red_color_percentage > blood_threshold

def checkGlare(img):
    print("[INFO] check glare")
    hist_full = cv2.calcHist([img],[0],None,[256],[0,256])
    maxWhite = 0
    clipCount = 0
    for i, h in enumerate(hist_full):
        if h > 0:
            maxWhite = i
        
        if i == len(hist_full) - 1:
            clipCount = h
            
    print("maxWhite", maxWhite, "clipCount", clipCount)
    
    return clipCount

In [4]:
class Point():
    def __init__(self, x, y):
        self.x = x
        self.y = y

In [6]:
class ImageProcessor:
    def __init__(self, rdt_type, rotated=False):
        config = json.load(open('configs/config.json'))
        
        self.rdt_type = rdt_type
        self.config = config[rdt_type]
        self.ref_img = cv2.imread('configs/%s.jpg'%self.config['REF_IMG'], 0)
        if rdt_type == 'experimental':
            self.ref_img_size = (216, 740)
        
        if rotated:
            self.ref_img = cv2.rotate(self.ref_img, cv2.ROTATE_90_CLOCKWISE)
            if rdt_type == 'experimental':
                self.ref_img_size = (740, 216)
            
        self.sift_detector = cv2.xfeatures2d.SIFT_create()
        self.matcher = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)
        ref_keypoints, ref_descriptors = self.sift_detector.detectAndCompute(
            self.ref_img, None)
        self.ref_keypoints = ref_keypoints 
        self.ref_descriptors = ref_descriptors
        
    def detectRDT(self, img, cnt=5):
        height, width, channel = img.shape
        p1 = (0, int(height * (1 - (self.config['VIEW_FINDER_SCALE_W']) / CROP_RATIO) / 2))
        p2 = (int(width - p1[0]), int(height - p1[1]))
        roi = img[p1[1]:p2[1], p1[0]:p2[0]]
        
        # img = roi
        # mask = np.zeros((height, width), np.uint8)
        # mask[p1[1]:p2[1], p1[0]: p2[0]] = 255
        keypoints, descriptors = self.sift_detector.detectAndCompute(img, None)
        
        
        # Matching
        if descriptors is None: 
            return None
        matches = self.matcher.knnMatch(self.ref_descriptors, descriptors, k=2)
        print('[INFO] Finish matching')
        # print('matches', matches)
        # Apply ratio test
        good = []
        if matches is None or len(matches) == 0 or len(matches[0]) < 2:
            return None
        for m,n in matches:
            if m.distance < 0.80*n.distance:
                good.append(m)


        #matchingImage = cv2.drawMatches(self.ref_img, self.ref_keypoints, img,
        #                               keypoints, good, None, flags=2)
        #matchingImage = cv2.cvtColor(matchingImage, cv2.COLOR_BGR2RGB)
        #plt.imshow(matchingImage)
        # plt.title('SIFT Brute Force matching')
        #plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)
        #plt.show()

        sum = 0
        distance = 0
        count = 0

        # store all the good matches as per Lowe's ratio test.
        img2 = None
        dst = None
        print('[INFO] matches')

        if len(good)> MIN_MATCH_COUNT:
            src_pts = np.float32([ self.ref_keypoints[m.queryIdx].pt for m in good ]).reshape(-1,1,2)
            dst_pts = np.float32([ keypoints[m.trainIdx].pt for m in good ]).reshape(-1,1,2)
            # print('src_pts', src_pts)
            # print('dst_pst', dst_pts)
            M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, cnt)
            print('[INFO] Finish finding Homography')
            # print('[INFO] M Matrix', M)
            # print('[INFO] mask', mask)
            matchesMask = mask.ravel().tolist()
            h,w = self.ref_img.shape
            pts = np.float32([ [0,0],[w-1,0],[w-1,h-1],[0,h-1] ]).reshape(-1,1,2)
            if M is None or M.size == 0:
                return None
            dst = cv2.perspectiveTransform(pts,M)
            # print('[INFO] dst transformation pts', dst)
            #img2 = np.copy(img)
            #img2 = cv2.polylines(img2,[np.int32(dst)],True,(255,0,0))
            #pts_box = cv2.minAreaRect(dst)
            #box = cv2.boxPoints(pts_box) # cv2.boxPoints(rect) for OpenCV 3.x
            #box = np.int0(box)
            #cv2.drawContours(img2,[box],0,(0,0,255),2)
            #print('[INFO] finish perspective transform')
            #print(box)
            #show_image(roi)
            #show_image(img2)

        else:
            print( "Not enough matches are found - {}/{}".format(len(good), MIN_MATCH_COUNT) )
            matchesMask = None
            return None

        draw_params = dict(matchColor = (255,0,0), # draw matches in green color
                   singlePointColor = None,
                   matchesMask = matchesMask, # draw only inliers
                   flags = 2)
        #img3 = cv2.drawMatches(self.ref_img,self.ref_keypoints,img,keypoints,good,None,**draw_params)
        #plt.imshow(img3)
        #plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)
        #plt.show()
        # show_image(img3)

        h, w  = self.ref_img.shape
        #refBoundary = np.float32([ [0,0], [0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
        refBoundary = np.float32([ [0,0],[w-1,0],[w-1,h-1],[0,h-1] ]).reshape(-1,1,2)
        # print('Refboundary', refBoundary)
        # print('Boundary', dst)
        M = cv2.getPerspectiveTransform(dst, refBoundary)
        # print('M matrixxxx', M)
        transformedImage = cv2.warpPerspective(img ,M, (self.ref_img.shape[1], self.ref_img.shape[0]))
        if self.rdt_type == 'experimental':
            transformedImage = cv2.resize(transformedImage, (self.ref_img_size[1], self.ref_img_size[0]))

        # show_image(roi)
        # show_image(transformedImage)
        transformedImage = cv2.cvtColor(transformedImage, cv2.COLOR_BGR2RGB)
        plt.imshow(transformedImage)
        plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False)
        plt.show()

        return dst 
    
    def interpretResult(self, img, boundary=None, enhanced=True):
        offset = 0
        
        cnt = 0
        while True:
            print('offset', offset)
            cropped_img = self.cropResultWindow(img, boundary, offset=offset)
            
            blood_percentage = calculateRedColorPercentage(cropped_img)

            clip_count = 0
            
            if enhanced:
                enhanced_img, clip_count = self.enhanceImage(cropped_img, (5, int(cropped_img.shape[1]/3)))
            else:
                enhanced_img = cropped_img

            enhanced_img_plt = cv2.cvtColor(enhanced_img, cv2.COLOR_BGR2RGB)

            h, w, _ = enhanced_img_plt.shape
            plt.imshow(enhanced_img_plt)
            plt.tick_params(top=False, bottom=False, left=False, right=False, labelleft=False, labelbottom=False) 
            plt.show()

            # Detect line location
            maxtab, numberOfLines, mintab = self.detectLinesWithPeak(enhanced_img)

            hls = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2HLS) 
            colLightness = np.mean(hls[:,:,1], axis=0)
            colLightness = [255] - colLightness

            maxtab_org = []
            mintab_org = []
            for t in maxtab:
                maxtab_org.append([t[0], colLightness[int(t[0])], t[2]])

            for t in mintab:
                mintab_org.append([t[0], colLightness[int(t[0])], t[2]])

            #print('Max Org', np.array(maxtab_org))
            #print('Min Org', np.array(mintab_org))

            results = self.detectLinesWithRelativeLocation(maxtab)
            
            break
        
        return results, blood_percentage, clip_count, maxtab, mintab, maxtab_org, mintab_org
        
    def enhanceImage(self, img, tile):
        print('[INFO] enhanceImage', img.shape)
        # show_image(img)
        #result = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)
        #img = cv2.GaussianBlur(img, (7,7), 0)
        #img = cv2.bilateralFilter(img,10,150,150)
        result = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
         # create a CLAHE object (Arguments are optional).
        clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=tile)
        channels = cv2.split(result)
        #channels[1] = cv2.bilateralFilter(channels[1],10,75,75)
        cv2.normalize(channels[1], channels[1], alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
        cl1 = clahe.apply(channels[1])
        channels[1] = cl1
        result = cv2.merge(channels)
        result = cv2.cvtColor(result, cv2.COLOR_HLS2RGB)
        
        clip_count = checkGlare(cl1)
        #result = cv2.cvtColor(result, cv2.RGB2RGBA)
        # show_image(result)
        return result, clip_count
    
    def cropResultWindow(self, img, boundary, offset=0):
        print('[INFO] cropResultWindow')
        # show_image(img)
        # show_image(self.fluRefImg)
        # print('Boundary shape', boundary.shape)
        # print('Boundary' , boundary)
        # print('Image shape', img.shape)
        h, w = self.ref_img.shape
        # img2 = cv2.polylines(img,[np.int32(boundary)],True,(255,0,0))
        # show_image(img2)
        #refBoundary = np.float32([ [0,0], [0,h-1],[w-1,h-1],[w-1,0]]).reshape(-1,1,2)
        refBoundary = np.float32([ [0,0],[w-1,0],[w-1,h-1],[0,h-1] ]).reshape(-1,1,2)
        # print('Refboundary', refBoundary)
        # print('Boundary', boundary)
        M = cv2.getPerspectiveTransform(boundary, refBoundary)
        transformedImage = cv2.warpPerspective(img,M, (self.ref_img.shape[1], self.ref_img.shape[0]))
        if self.rdt_type == 'experimental':
            transformedImage = cv2.resize(transformedImage, (self.ref_img_size[1], self.ref_img_size[0]))
        # show_image(transformedImage)
        # TODO: this is where things went wrong, it cannot perform perspective transform

        tl = Point(self.config['FIDUCIAL_TO_RESULT_WINDOW_OFFSET'] - self.config['RESULT_WINDOW_RECT_HEIGHT']/2.0 - offset, 
                   self.config['RESULT_WINDOW_RECT_WIDTH_PADDING'])
        br = Point(self.config['FIDUCIAL_TO_RESULT_WINDOW_OFFSET'] + self.config['RESULT_WINDOW_RECT_HEIGHT']/2.0 - offset,
                   transformedImage.shape[0] - self.config['RESULT_WINDOW_RECT_WIDTH_PADDING'])
                   
        if tl is None and br is None:
            return None
        # print('[INFO] tl, br', tl.x, tl.y, br.x, br.y)
        cropResultWindow = transformedImage[int(tl.y): int(br.y), int(tl.x):int(br.x)]
        # print('[INFO] shape', cropResultWindow.shape, transformedImage.shape, img.shape)
        # show_image(cropResultWindow)
        # if (cropResultWindow.shape[0] > 0 and cropResultWindow.shape[1] > 0):
        #     cropResultWindow.reshape((RESULT_WINDOW_RECT_HEIGHT, self.fluRefImg.shape[0] - 2 * RESULT_WINDOW_RECT_WIDTH_PADDING))
        #     show_image(transformedImage)
        return cropResultWindow
    
    def detectLinesWithPeak(self, img):
        #variables = {}
        #with open('variables/variables.json') as json_file:
        #    variables = json.load(json_file)
        
        print(img.shape)
        # show_image(img)
        hls = cv2.cvtColor(img, cv2.COLOR_BGR2HLS) 
        # show_image(hls)
        # hls1 = cv2.cvtColor(img, cv2.COLOR_RGB2HLS) 
        # show_image(hls1)
        # HSL so only take the L channel to distinguish lines
        # print('[INFO] result img shape', img.shape)
        colR = np.mean(img[:,:,0], axis=0)
        colG = np.mean(img[:,:,1], axis=0)
        colB = np.mean(img[:,:,2], axis=0)

        colLightness = np.mean(hls[:,:,1], axis=0)
        colHue = np.mean(hls[:,:,0], axis=0)
        colSaturation = np.mean(hls[:,:,2], axis=0)
        # plt.plot(colLightness)
        # plt.show()

        # plt.plot(colHue)
        # plt.show()

        # plt.plot(colSaturation)
        # plt.show()
        # print('[INFO] avgLightness shape', colLightness.shape)
        # Inverse the L channel so that the lines will be detected as peak, not bottom like the original array
        colLightness = [255] - colLightness
        plt.figure(figsize=(5, 1.5))
        plt.tick_params(
                axis='x',          # changes apply to the x-axis
                which='both',      # both major and minor ticks are affected
                bottom=False,      # ticks along the bottom edge are off
                top=False,         # ticks along the top edge are off
                labelbottom=False) # labels along the bottom edge are off
        plt.ylim(0, 255)
        plt.ylabel("Lightness (L)", fontsize=12)
        plt.yticks([0, 255])
        plt.plot(range(0, len(colLightness)), [255] - colLightness, '-o')
        plt.show()
        
        #plt.figure(figsize=(5, 1.5))
        #plt.tick_params(
        #        axis='x',          # changes apply to the x-axis
        #        which='both',      # both major and minor ticks are affected
        #        bottom=False,      # ticks along the bottom edge are off
        #        top=False,         # ticks along the top edge are off
        #        labelbottom=False) # labels along the bottom edge are off
        #plt.yticks([0, 180])
        #plt.ylim(0, 180)
        #plt.ylabel("Hue (H)", fontsize=12)
        #plt.plot(range(0, len(colHue)), colHue, '-o')
        #plt.show()
        # print('[INFO] lightness shape', colLightness.shape)
        # Find peak and peak should correspond to lines
        maxtab, mintab = peakdet(colLightness, self.config['CONTROL_INTENSITY_PEAK_THRESHOLD'])
        print('Max', maxtab)
        #print('Min', mintab)
        #print('Min', mintab)
        #print('Total strip count', len(maxtab))

        return maxtab, len(maxtab), mintab
    
        
    def detectLinesWithRelativeLocation(self, maxtab):
        results = [False]*3
        for col, val, width in maxtab:
            if col > self.config['TOP_LINE_POSITION'] - self.config['LINE_SEARCH_WIDTH'] and col < self.config['TOP_LINE_POSITION'] + self.config['LINE_SEARCH_WIDTH']:
                results[0] = True
            if col > self.config['MIDDLE_LINE_POSITION'] - self.config['LINE_SEARCH_WIDTH'] and col < self.config['MIDDLE_LINE_POSITION'] + self.config['LINE_SEARCH_WIDTH']:
                results[1] = True
            if col > self.config['BOTTOM_LINE_POSITION'] - self.config['LINE_SEARCH_WIDTH'] and col < self.config['BOTTOM_LINE_POSITION'] + self.config['LINE_SEARCH_WIDTH']+10:
                results[2] = True
        return results
        
    